### Combining data from OPUS for model training 

In [ ]:
"""

import pandas as pd

# Пути к файлам (замените на свои локальные пути)
ru_file = "TED2020.en-ru.ru"
en_file = "TED2020.en-ru.en"
output_file = "TED2020_translations.xlsx"

# Читаем файлы построчно
with open(ru_file, "r", encoding="utf-8") as f:
    ru_sentences = f.readlines()

with open(en_file, "r", encoding="utf-8") as f:
    en_sentences = f.readlines()

# Проверяем, что количество строк совпадает
if len(ru_sentences) != len(en_sentences):
    print("Ошибка: Файлы содержат разное количество строк!")
else:
    print(f"Загружено {len(ru_sentences)} пар предложений.")

# Создаём DataFrame
df = pd.DataFrame({"Russian": [s.strip() for s in ru_sentences], 
                   "English": [s.strip() for s in en_sentences]})

# Сохраняем в Excel
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"🎉 Данные успешно сохранены в файл {output_file}")"

"""


### Data preparing 

In [ ]:
"""

import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("TED2020_translations.xlsx")

# Split into train, validation, and test sets
train, temp = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Save datasets as Excel files
train.to_excel("train.xlsx", index=False, engine="openpyxl")
val.to_excel("val.xlsx", index=False, engine="openpyxl")
test.to_excel("test.xlsx", index=False, engine="openpyxl")

print(f"Data prepared and saved: train({len(train)}), val({len(val)}), test({len(test)})")"

"""

Data prepared and saved: train(312011), val(39001), test(39002)


### Cleaning the data 

In [9]:
import pandas as pd

# Load training and validation data from Excel
train_data = pd.read_excel("train.xlsx", engine="openpyxl")
val_data = pd.read_excel("val.xlsx", engine="openpyxl")

# Drop NaN values and convert everything to strings
train_data = train_data.dropna().astype(str)
val_data = val_data.dropna().astype(str)

# Save cleaned data (optional step for debugging)
train_data.to_excel("train_cleaned.xlsx", index=False, engine="openpyxl")
val_data.to_excel("val_cleaned.xlsx", index=False, engine="openpyxl")

print("Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx")

Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx


### Train the MarianMT Model

In [2]:
!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 56.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 74.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 70.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 81.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 85.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 72.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install transformers[torch] accelerate>=0.26.0

In [2]:
### Small dataset for fast training

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")

# Take only a subset of data (e.g., 10,000 samples)
subset_size = 10000
df = df.sample(n=subset_size, random_state=42)  # Randomly select 10,000 rows

# Split into train/validation
train, val = train_test_split(df, test_size=0.2, random_state=42)

# Save the smaller dataset
train.to_excel("train_small.xlsx", index=False, engine="openpyxl")
val.to_excel("val_small.xlsx", index=False, engine="openpyxl")

print(f"✅ Prepared smaller dataset with {len(train)} training and {len(val)} validation samples.")

✅ Prepared smaller dataset with 8000 training and 2000 validation samples.


In [ ]:
### Second try training with small dataset

In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load the smaller dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load pre-trained MarianMT model
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments (optimized for faster training)
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model",  # ✅ Save model in a clear directory
    num_train_epochs=2,  # ✅ Reduce epochs for faster training
    per_device_train_batch_size=16,  # ✅ Increase batch size to speed up training
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1  # ✅ Keep only the latest checkpoint
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model in a clear directory
model.save_pretrained("custom_translation_model")
tokenizer.save_pretrained("custom_translation_model")

print("🎉 Training completed! Model saved in 'custom_translation_model'")


2025-03-18 19:44:58.328579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 19:44:58.341621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742327098.356305   18486 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742327098.360866   18486 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742327098.373535   18486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.795100,0.730192
2,0.691100,0.678546


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


🎉 Training completed! Model saved in 'custom_translation_model'


In [ ]:
### Second try, others hyperparametrs

In [2]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load the smaller dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load pre-trained MarianMT model
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Optimized Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v2",  # ✅ New folder for improved model
    num_train_epochs=4,  # ✅ More epochs for better training
    per_device_train_batch_size=32,  # ✅ Larger batch size (if enough memory)
    gradient_accumulation_steps=4,  # ✅ Effective larger batch size if memory is limited
    learning_rate=3e-5,  # ✅ Slower learning rate for stability
    weight_decay=0.01,  # ✅ Regularization to prevent overfitting
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained("custom_translation_model_v2")
tokenizer.save_pretrained("custom_translation_model_v2")

print("Training completed! Model saved in 'custom_translation_model_v2'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.167600,1.126219
2,1.112500,0.946095
3,0.921500,0.869714


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! Model saved in 'custom_translation_model_v2'


In [ ]:
### Third model

In [7]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function with label smoothing support
def preprocess_function(examples):
    model_inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v1_5",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True  # Required for generation-based eval
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train
trainer.train()

# Save model
model.save_pretrained("custom_translation_model_v1_5")
tokenizer.save_pretrained("custom_translation_model_v1_5")

print("✅ Improved model (v1.5) training complete and saved.")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.320800,0.315128
2,0.291100,0.315201
3,0.277000,0.316235


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


✅ Improved model (v1.5) training complete and saved.


In [ ]:
### With EarlyStopping

In [8]:
import pandas as pd
from transformers import (
    MarianMTModel,
    MarianTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset

# 📌 Step 1: Load the dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# 📌 Step 2: Load model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 📌 Step 3: Preprocessing
def preprocess_function(examples):
    model_inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# 📌 Step 4: Training arguments with EarlyStopping setup
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v1_5_earlystop",
    num_train_epochs=10,  # max limit, early stopping will stop earlier if needed
    per_device_train_batch_size=32,
    learning_rate=3e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True,
    load_best_model_at_end=True,  # ⬅️ restores best model
    metric_for_best_model="eval_loss",  # based on validation loss
    greater_is_better=False
)

# 📌 Step 5: Trainer with EarlyStoppingCallback
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # ⏹️ stop after 2 non-improving epochs
)

# 📌 Step 6: Train and save
trainer.train()

model.save_pretrained("custom_translation_model_v1_5_earlystop")
tokenizer.save_pretrained("custom_translation_model_v1_5_earlystop")

print("✅ Training complete! Best model saved in 'custom_translation_model_v1_5_earlystop'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.321400,0.315831
2,0.289000,0.317355
3,0.266500,0.320317


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


✅ Training complete! Best model saved in 'custom_translation_model_v1_5_earlystop'


In [2]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# 🔹 Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# 🔹 Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# 🔹 Загружаем модель v1_5_earlystop
model_path = "custom_translation_model_v1_5_earlystop"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# 🔹 Используем GPU если доступно
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 🔹 Функция пакетного перевода
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# 🔹 Разбиваем на батчи
batch_size = 16
sentences = df["Sentence"].astype(str).tolist()
translations = [
    translation
    for i in range(0, len(sentences), batch_size)
    for translation in batch_translate(sentences[i:i+batch_size], model, tokenizer)
]

# 🔹 Сохраняем результат
df["Translation_v1_5"] = translations
df.to_excel("translated_assembly_WER_v1_5.xlsx", index=False, engine="openpyxl")

print("✅ Перевод завершён и сохранён в 'translated_assembly_WER_v1_5.xlsx'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Перевод завершён и сохранён в 'translated_assembly_WER_v1_5.xlsx'


In [13]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback
from datasets import Dataset

# ✅ Загружаем данные
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# ✅ Конвертируем в Dataset формата Hugging Face
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# ✅ Загружаем модель MarianMT (Transformer)
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# ✅ Функция токенизации
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# ✅ Применяем токенизацию
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# ✅ Параметры обучения
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_final",  # 🏁 Финальная папка для модели
    num_train_epochs=10,  # 🔥 Дольше обучение
    per_device_train_batch_size=16,  # ⚖️ Оптимальный размер батча
    gradient_accumulation_steps=4,  # 🧠 Нужен, если памяти мало
    learning_rate=2e-5,  # 🚀 Стабильный лернинг рейт
    weight_decay=0.01,  # 🛡️ Защита от переобучения
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1,  # 🗑️ Храним только последний чекпойнт
    fp16=True,  # ⚡ Используем Mixed Precision для ускорения
    load_best_model_at_end=True,  # 🏆 Загружаем лучшую модель
    metric_for_best_model="eval_loss",
    greater_is_better=False,
)

# ✅ Добавляем Early Stopping (если loss не падает 2 эпохи — стоп)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 🚀 Запуск обучения
trainer.train()

# ✅ Сохранение модели
model.save_pretrained("custom_translation_model_final")
tokenizer.save_pretrained("custom_translation_model_final")

print("✅ Финальное обучение завершено! Модель сохранена в 'custom_translation_model_final'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.254700,1.032827
2,0.930700,0.867239
3,0.852400,0.793226
4,0.785000,0.750331
5,0.736400,0.723534
6,0.700200,0.705162
7,0.690700,0.692900
8,0.671500,0.684287
9,0.655300,0.680194
10,0.643400,0.678575


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


✅ Финальное обучение завершено! Модель сохранена в 'custom_translation_model_final'


In [16]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback
from datasets import Dataset

# ✅ Загружаем данные
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# ✅ Конвертируем в Dataset формата Hugging Face
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# ✅ Инициализируем модель MarianMT с нуля (без предварительных чекпойнтов)
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# ✅ Функция токенизации
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# ✅ Применяем токенизацию
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# ✅ Параметры обучения с гиперпараметрами для стабильности и эффективности
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_v3",  # Папка для сохранения модели
    num_train_epochs=5,  # Количество эпох
    per_device_train_batch_size=16,  # Размер батча
    gradient_accumulation_steps=4,  # Накопление градиентов (если памяти не хватает)
    learning_rate=2e-5,  # Меньший лернинг рейт для стабильности
    weight_decay=0.01,  # Регуляризация для предотвращения переобучения
    evaluation_strategy="epoch",  # Оценка каждый эпох
    save_strategy="epoch",  # Сохранение модели каждый эпох
    logging_steps=50,  # Логгирование каждую 50-ю итерацию
    save_total_limit=1,  # Хранить только последний чекпойнт
    fp16=True,  # Используем mixed precision для ускорения
    load_best_model_at_end=True,  # Загружаем лучший чекпойнт по валид. потере
    metric_for_best_model="eval_loss",  # Метрика для выбора лучшей модели
    greater_is_better=False,  # Чем меньше eval_loss, тем лучше
)

# ✅ Добавляем Early Stopping (если модель не улучшилась 2 эпохи — остановить обучение)
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 🚀 Запуск обучения с нуля
trainer.train()

# ✅ Сохранение модели после обучения
model.save_pretrained("custom_translation_model_v3")
tokenizer.save_pretrained("custom_translation_model_v3")

print("✅ Обучение завершено! Модель сохранена в 'custom_translation_model_v3'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.263600,1.046986
2,0.954200,0.892963
3,0.887900,0.831931
4,0.841900,0.803147
5,0.811500,0.794744


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


✅ Обучение завершено! Модель сохранена в 'custom_translation_model_v3'


In [4]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# Загружаем обе модели
model1_name = "custom_translation_model"
model2_name = "custom_translation_model_v2"

tokenizer1 = MarianTokenizer.from_pretrained(model1_name)
tokenizer2 = MarianTokenizer.from_pretrained(model2_name)

model1 = MarianMTModel.from_pretrained(model1_name)
model2 = MarianMTModel.from_pretrained(model2_name)

# Проверяем, есть ли GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model1.to(device)
model2.to(device)

# Функция для пакетного перевода
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Разбиваем текст на batch-и по 16 предложений
batch_size = 16

df["Translation_Model_1"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model1, tokenizer1)
]

df["Translation_Model_2"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model2, tokenizer2)
]

# Сохраняем результат в новый Excel-файл
output_file = "translated_assembly_WER.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Перевод завершён! Результаты сохранены в '{output_file}'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Перевод завершён! Результаты сохранены в 'translated_assembly_WER.xlsx'


In [2]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# Загружаем обе модели
model1_name = "custom_translation_model"
model2_name = "custom_translation_model_v2"

tokenizer1 = MarianTokenizer.from_pretrained(model1_name)
tokenizer2 = MarianTokenizer.from_pretrained(model2_name)

model1 = MarianMTModel.from_pretrained(model1_name).to("cuda")  # ✅ Сразу на GPU
model2 = MarianMTModel.from_pretrained(model2_name).to("cuda")  # ✅ Сразу на GPU

# Функция для пакетного перевода
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to("cuda")
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Разбиваем текст на batch-и по 16 предложений
batch_size = 16

df["Translation_Model_1"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model1, tokenizer1)
]

df["Translation_Model_2"] = [
    trans for i in range(0, len(df), batch_size)
    for trans in batch_translate(df["Sentence"].astype(str).tolist()[i:i + batch_size], model2, tokenizer2)
]

# Сохраняем результат в новый Excel-файл
output_file = "translated_assembly_WER.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Перевод завершён! Результаты сохранены в '{output_file}'")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


✅ Перевод завершён! Результаты сохранены в 'translated_assembly_WER.xlsx'


In [ ]:
### First try of training

In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load cleaned training and validation data from Excel
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

# Convert Pandas DataFrame to Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load the MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function for tokenization
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization with batch processing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained(".")
tokenizer.save_pretrained(".")

print("Training completed! Model saved")

2025-03-18 16:31:51.257754: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 16:31:51.270838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742315511.285192   15960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742315511.289643   15960 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742315511.302109   15960 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/309030 [00:00<?, ? examples/s]

Map:   0%|          | 0/38613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.443300,0.363478
2,0.367000,0.337283
3,0.293500,0.327943


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! Model saved


In [ ]:
### FINAL TRAINING - !!!

In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback
from datasets import Dataset

# Load the full training and validation data
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load the MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization to datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model_full",
    num_train_epochs=10,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=50,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Create the trainer with EarlyStopping
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Start training
trainer.train()

# Save the final model
model.save_pretrained("custom_translation_model_full")
tokenizer.save_pretrained("custom_translation_model_full")

print("Training completed. Model saved to 'custom_translation_model_full'")

INFO: PyTorch version 2.6.0 available.
INFO: TensorFlow version 2.19.0 available.
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/309030 [00:00<?, ? examples/s]

Map:   0%|          | 0/38613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.472300,0.432152
1,0.420500,0.390194
2,0.410700,0.371440
3,0.384200,0.361543
4,0.365300,0.354061
5,0.354600,0.349739
6,0.359700,0.345944
7,0.341400,0.343992
8,0.331100,0.342949
9,0.332500,0.342440


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


Training completed. Model saved to 'custom_translation_model_full'


In [6]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load the input dataset
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Load the final trained translation model
model_path = "custom_translation_model_full"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Batch translation function
def batch_translate(sentences, model, tokenizer):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(**inputs)
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Translate in batches
batch_size = 16
sentences = df["Sentence"].astype(str).tolist()
translations = [
    translation
    for i in range(0, len(sentences), batch_size)
    for translation in batch_translate(sentences[i:i + batch_size], model, tokenizer)
]

# Save the translations
df["Translation"] = translations
df.to_excel("translated_assembly_WER_full.xlsx", index=False, engine="openpyxl")

print("Translation completed successfully. Output saved to 'translated_assembly_WER_full.xlsx'.")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translation completed successfully. Output saved to 'translated_assembly_WER_full.xlsx'.


In [ ]:
### Translation with beam=5

In [7]:
import pandas as pd
import torch
from transformers import MarianMTModel, MarianTokenizer

# Load the input Excel file
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Ensure the sentence column exists
if "Sentence" not in df.columns:
    raise ValueError("The file must contain a column named 'Sentence'.")

# Load your trained translation model
model_path = "custom_translation_model_full"
tokenizer = MarianTokenizer.from_pretrained(model_path)
model = MarianMTModel.from_pretrained(model_path)

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Translation function with beam search
def batch_translate(sentences, model, tokenizer, num_beams=5):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated = model.generate(
        **inputs,
        num_beams=num_beams,
        early_stopping=True
    )
    return tokenizer.batch_decode(translated, skip_special_tokens=True)

# Translate in batches
batch_size = 16
sentences = df["Sentence"].astype(str).tolist()
translations = [
    translation
    for i in range(0, len(sentences), batch_size)
    for translation in batch_translate(sentences[i:i + batch_size], model, tokenizer, num_beams=5)
]

# Save results to new column and file
df["Translation_beam5"] = translations
df.to_excel("translated_assembly_WER_beam5.xlsx", index=False, engine="openpyxl")

print("Translation completed using beam search. Output saved to 'translated_assembly_WER_beam5.xlsx'.")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translation completed using beam search. Output saved to 'translated_assembly_WER_beam5.xlsx'.


In [ ]:
### Training with mBART with small dataset

In [3]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments, EarlyStoppingCallback
from datasets import Dataset

# Load the small train and validation sets
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load mBART model and tokenizer
model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Set source and target language
tokenizer.src_lang = "ru_RU"
target_lang = "en_XX"

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length", return_tensors="pt")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Training settings
training_args = Seq2SeqTrainingArguments(
    output_dir="mbart_translation_tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    num_train_epochs=4,  # но будет ранняя остановка
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_steps=200,
    label_smoothing_factor=0.1,
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=torch.cuda.is_available()
)


# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train
trainer.train()

# Save the model
model.save_pretrained("mbart_translation_tuned")
tokenizer.save_pretrained("mbart_translation_tuned")

print("mBART training on small dataset completed.")

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_72207/2351858815.py:56: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss
1,2.461100,2.448836
2,2.372900,2.400036
3,2.309700,2.417953
4,2.257800,2.405698


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


mBART training on small dataset completed.


In [ ]:
### Translaiting with small dataset

In [2]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the test file
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Ensure the sentence column exists
if "Sentence" not in df.columns:
    raise ValueError("The file must contain a column named 'Sentence'.")

# Load trained mBART model and tokenizer
model_path = "mbart_translation_small"
tokenizer = MBart50TokenizerFast.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)

# Set source and target languages
tokenizer.src_lang = "ru_RU"
target_lang_code = "en_XX"

# Use GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Translation function with beam search
def batch_translate(sentences, tokenizer, model, batch_size=16, num_beams=5):
    translations = []
    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
        translated = model.generate(
            **inputs,
            forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang_code),
            num_beams=num_beams,
            max_length=128,
            early_stopping=True
        )
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        translations.extend(decoded)
    return translations

# Translate all sentences
sentences = df["Sentence"].astype(str).tolist()
translations = batch_translate(sentences, tokenizer, model)

# Save translations
df["Translation_mBART"] = translations
df.to_excel("translated_assembly_WER_mbart.xlsx", index=False, engine="openpyxl")

print("Translation completed using mBART. Output saved to 'translated_assembly_WER_mbart.xlsx'.")

Translation completed using mBART. Output saved to 'translated_assembly_WER_mbart.xlsx'.


In [ ]:
### Training mBART with full dataset

In [9]:
import pandas as pd
import torch
from transformers import (
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback
)
from datasets import Dataset

# Load full training and validation data
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load tokenizer (original name is fine here)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "en_XX"

# Load model directly from checkpoint
model = MBartForConditionalGeneration.from_pretrained("mbart_translation_full/checkpoint-9656")

# Preprocessing function using `text_target` (modern API)
def preprocess_function(examples):
    return tokenizer(
        examples["Russian"],
        text_target=examples["English"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )

# Tokenize datasets
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="mbart_translation_full",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=50,
    save_total_limit=1,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Resume training
trainer.train(resume_from_checkpoint=True)

# Save final model
model.save_pretrained("mbart_translation_full")
tokenizer.save_pretrained("mbart_translation_full")

print("Training complete. Model saved in 'mbart_translation_full'")

Map:   0%|          | 0/309030 [00:00<?, ? examples/s]

Map:   0%|          | 0/38613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_72207/294763683.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Epoch,Training Loss,Validation Loss
2,0.195000,0.229390
3,0.176800,0.234234


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Training complete. Model saved in 'mbart_translation_full'


In [ ]:
### Translation using mBART

In [1]:
import pandas as pd
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the Excel file
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Check that 'Sentence' column exists
if "Sentence" not in df.columns:
    raise ValueError("The file does not contain a 'Sentence' column. Please check the structure.")

# Load the trained mBART model and tokenizer
model_path = "mbart_translation_full"
tokenizer = MBart50TokenizerFast.from_pretrained(model_path)
model = MBartForConditionalGeneration.from_pretrained(model_path)

# Set source and target languages
tokenizer.src_lang = "ru_RU"
tokenizer.tgt_lang = "en_XX"

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Translation function
def batch_translate(sentences, model, tokenizer, num_beams=10):
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    translated_tokens = model.generate(
        **inputs,
        num_beams=num_beams,
        max_length=128,
        early_stopping=True
    )
    return tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)

# Perform translation in batches
batch_size = 16
sentences = df["Sentence"].astype(str).tolist()
translations = [
    translation
    for i in range(0, len(sentences), batch_size)
    for translation in batch_translate(sentences[i:i + batch_size], model, tokenizer)
]

# Add translations to DataFrame
df["Translation_mBART"] = translations

# Save to new Excel file
output_file = "translated_assembly_WER_mBART_bean10.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"Translation completed and saved to '{output_file}'")

2025-03-26 09:55:29.396922: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 09:55:29.477641: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742982929.515803      22 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742982929.528359      22 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742982929.601800      22 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Translation completed and saved to 'translated_assembly_WER_mBART_bean10.xlsx'
